In [4]:
pip install azure-mgmt-compute

     ---------------------------------------- 4.9/4.9 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 41.7/41.7 kB ? eta 0:00:00
     -------------------------------------- 174.5/174.5 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install azureml

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install azureml-core

     ---------------------------------------- 3.3/3.3 MB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 1.8 MB/s eta 0:00:00
     -------------------------------------- 148.1/148.1 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 90.0/90.0 kB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 2.8/2.8 MB 2.1 MB/s eta 0:00:00
     -------------------------------------- 780.2/780.2 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 40.5/40.5 kB 2.0 MB/s eta 0:00:00
     -------------------------------------- 965.9/965.9 kB 2.1 MB/s eta 0:00:00
     -------------------------------------- 55.5/55.5 kB 963.8 kB/s eta 0:00:00
     ---------------------------------------- 86.8/86.8 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 61.1/61.1 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 40

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\anaconda3\\Lib\\site-packages\\win32\\_win32sysloader.pyd'
Consider using the `--user` option or check the permissions.



In [11]:
pip install --user azureml-core

  Using cached azureml_core-1.52.0-py3-none-any.whl (3.3 MB)
  Using cached knack-0.10.1-py3-none-any.whl (61 kB)
  Using cached azure_mgmt_containerregistry-10.1.0-py3-none-any.whl (1.7 MB)
  Using cached azure_mgmt_authorization-3.0.0-py3-none-any.whl (965 kB)
  Using cached msal-1.22.0-py2.py3-none-any.whl (90 kB)
  Using cached SecretStorage-3.3.3-py3-none-any.whl (15 kB)
  Using cached contextlib2-21.6.0-py2.py3-none-any.whl (13 kB)
  Using cached azure_mgmt_resource-22.0.0-py3-none-any.whl (2.4 MB)
  Using cached msrestazure-0.6.4-py2.py3-none-any.whl (40 kB)
  Using cached docker-6.1.3-py3-none-any.whl (148 kB)
  Using cached azure_graphrbac-0.61.1-py2.py3-none-any.whl (141 kB)
  Using cached azure_mgmt_keyvault-10.2.2-py3-none-any.whl (780 kB)
  Using cached azure_mgmt_storage-21.0.0-py3-none-any.whl (2.8 MB)
  Using cached argcomplete-2.1.2-py3-none-any.whl (37 kB)
  Using cached ndg_httpsclient-0.5.1-py3-none-any.whl (34 kB)
  Using cached msrest-0.7.1-py3-none-any.whl (85 kB

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
pip install azureml-sdk

     ---------------------------------------- 8.6/8.6 MB 2.1 MB/s eta 0:00:00
     -------------------------------------- 137.2/137.2 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 20.6/20.6 MB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 38.2/38.2 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 69.5/69.5 kB 759.4 kB/s eta 0:00:00
     ------------------------------------ 313.1/313.1 kB 807.7 kB/s eta 0:00:00
     -------------------------------------- 247.1/247.1 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 31.7/31.7 MB 1.9 MB/s eta 0:00:00
     -------------------------------------- 151.6/151.6 kB 1.8 MB/s eta 0:00:00
     -------------------------------------- 905.7/905.7 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 16.0/16.0 MB 1.3 MB/s eta 0:00:00
     ---------------

In [15]:
import azure.core
from azureml.core import Workspace,Environment,Experiment,ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [28]:
resource_name = "rg_project_ml"
workspace_name = "w_project_ml"
subscriptionID = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" #f2813f24-bf98-421d-b7cb-554e68274436
aml_compute_target = "projectml" 
experiment_name= 'credit_card_project1'

In [6]:
# Now create Workspace
try:
    ws=Workspace.from_config()
    print('Workspace is already exist')
except:
    ws=Workspace.create(workspace_name, 
                    resource_group=resource_name,
                    create_resource_group=True,
                    subscription_id=subscriptionID,
                    location="East US")
    ws.write_config('.azureml')

Workspace is already exist


In [12]:
# Create Compute Target
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :",aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D4_v3",
                                                                min_nodes = 1, 
                                                                max_nodes = 4,
                                               idle_seconds_before_scaledown=10000)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached now")

This Compute Target already exist.
Azure Machine Learning Compute attached now


In [29]:
# Create Experiment
exp = Experiment(ws,experiment_name)

In [30]:
# Create environment to execute your code
env = Environment.from_conda_specification(name="azure_ml",file_path="./envfile.yml")
config=ScriptRunConfig(source_directory="./",script="credit_card_detect.py",compute_target=aml_compute_target,environment=env)
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)

Submitting c:\Users\User\Untitled Folder 1 directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


RunId: credit_card_project1_1688443712_afd824f1
Web View: https://ml.azure.com/runs/credit_card_project1_1688443712_afd824f1?wsid=/subscriptions/f2813f24-bf98-421d-b7cb-554e68274436/resourcegroups/rg_project_ml/workspaces/w_project_ml&tid=5585330f-9739-4fc3-9558-6526f2590536

Streaming user_logs/std_log.txt

Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 5.702739953994751 seconds

Execution Summary
RunId: credit_card_project1_1688443712_afd824f1
Web View: https://ml.azure.com/runs/credit_card_project1_1688443712_afd824f1?wsid=/subscriptions/f2813f24-bf98-421d-b7cb-554e68274436/resourcegroups/rg_project_ml/workspaces/w_project_ml&tid=5585330f-9739-4fc3-9558-6526f2590536



{'runId': 'credit_card_project1_1688443712_afd824f1',
 'target': 'projectml',
 'status': 'Completed',
 'startTimeUtc': '2023-07-04T04:09:40.894265Z',
 'endTimeUtc': '2023-07-04T04:15:33.922972Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '2c6a0ed6-3cdb-44ca-9358-b83762f91be6',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'credit_card_detect.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'projectml',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': '

In [31]:
#Register model in workspace
from azureml.core import Model
model = Model.register(ws,model_path="./credit_card.pkl",model_name="Credit_Fraud_Model")

Registering model Credit_Fraud_Model


In [32]:
env.environment_variables

{'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE'}

In [33]:
model = Model(ws,"Credit_Fraud_Model")

In [34]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

myenv=Environment(name="credit_fraud-env")
conda_packages = ['numpy','pandas']
pip_packages = ['azureml-sdk','azureml-defaults','scikit-learn']
mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages, python_version='3.6.2')
myenv.python.conda_dependencies=mycondaenv
myenv.register(workspace=ws)



{
    "assetId": "azureml://locations/eastus/workspaces/af792426-5df2-445b-9938-cdccb6b0bc5f/environments/credit_fraud-env/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230620.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "credit_fraud-env",
  

In [35]:
inference_config = InferenceConfig(entry_script='model_score.py',source_directory='.',environment=myenv)

In [36]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1)

In [37]:
from azureml.core.model import Model
# Deploying the model
service = Model.deploy(ws,"creditcarddetection", #This is endpoint name
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aciconfig,
                           #deployment_target=predenv,
                           overwrite=True)
service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)

C:\Users\User\AppData\Local\Temp\ipykernel_26032\1110386421.py:3: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws,"creditcarddetection", #This is endpoint name


HttpResponseError: Operation could not be completed within the specified time.
RequestId:7de31c40-c01e-0099-4b30-aead12000000
Time:2023-07-04T04:29:58.0871108Z
ErrorCode:OperationTimedOut
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>OperationTimedOut</Code><Message>Operation could not be completed within the specified time.
RequestId:7de31c40-c01e-0099-4b30-aead12000000
Time:2023-07-04T04:29:58.0871108Z</Message></Error>